In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution3D, MaxPooling3D

from keras.optimizers import SGD, RMSprop
from keras.utils import np_utils, generic_utils

import theano
import os
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

import numpy as np
import pandas as pd
import cv2
import tqdm
import heapq
import datetime
import glob
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import TensorBoard
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split



TRAIN_PATH = '/content/drive/My Drive/UCF-CV_Project/Training_set/Training'
TEST_PATH = '/content/drive/My Drive/UCF-CV_Project/Testing_set'



# image specification
img_rows,img_cols,img_depth= 112,112, 40


# Training data

X_tr = []           # variable to store entire dataset


label=[]

for index, category in enumerate(os.listdir(TRAIN_PATH)):

      print(index, category)
      category_path = os.path.join(TRAIN_PATH, category)
      for video in os.listdir(category_path):
          video_path = os.path.join(category_path, video)
          frames = []
          cap = cv2.VideoCapture(video_path)
          num_of_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
          #ba5od el videos el 3adad el frames bt3etha akbar men 40
          if(num_of_frames < 40):continue
          #lw 3adad el frames kbar men 40 ba5od kol ad eh
          num = max(1, num_of_frames // 40)
          frame_idx = 0
          taken = 0
          while(1):
              ret, frame = cap.read()
              if(ret == False):
                break
              # Lw howa da el frame el el mfroud a5do fa ha5do
              if frame_idx % num == 0:
                  frame=cv2.resize(frame,(img_rows,img_cols),interpolation=cv2.INTER_AREA)
                  gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

                  frames.append(gray)
                  taken += 1
              if(taken == 40):
                break
              frame_idx += 1

          cap.release()
          cv2.destroyAllWindows()
          input=np.array(frames)
          X_tr.append(input)
          label.append(index)
      print(len(X_tr))
lebel = np.array(label)
X_tr_array = np.array(X_tr)   # convert the frames read into array
num_samples = len(X_tr_array)

print(num_samples)

In [ ]:
print(label)

In [ ]:
#concatination , column gamb column
train_data = [X_tr_array,label]

(X_train, y_train) = (train_data[0],train_data[1])

X_train = np.array(X_train)
#3ashan el gray scale maloush channel fana zawedtelo channel we ratebt el dimensions 
X_train = np.expand_dims(X_train, axis = 0)
print('X_Train shape:', X_train.shape)

print(X_train.shape)
X_train = X_train.transpose((1 , 3, 4 , 2, 0))
print(X_train.shape)


In [ ]:

print(X_train.shape)



patch_size = 40    # img_depth or number of frames used for each video

print(X_train.shape, 'train samples')

# CNN Training parameters
#bad5al 8 videos 3ala el network
batch_size = 8
nb_classes = 5
nb_epoch = 100

# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, nb_classes)



# Pre-processing
X_train = X_train.astype('float32')
X_train -= np.mean(X_train)
X_train /=np.max(X_train)

(478, 112, 112, 40, 1)
(478, 112, 112, 40, 1) train samples


In [ ]:
X_train.shape[1:]

(112, 112, 40, 1)

In [ ]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Flatten
from keras.layers.convolutional import Convolution3D, MaxPooling3D, ZeroPadding3D
from keras.optimizers import SGD


model = Sequential()
# 64 filters, b3den size el filter , el video el da5el 
model.add(Convolution3D(64,3, 3, 3, input_shape=(img_rows, img_rows, 40, 1), activation='relu'))



model.add(MaxPooling3D(pool_size=(3, 3, 3)))

model.add(Dropout(0.5))

model.add(Flatten())

model.add(Dense(128, init='normal', activation='relu'))

model.add(Dropout(0.5))

model.add(Dense(nb_classes,init='normal'))

model.add(Activation('softmax'))

#model.compile(loss='categorical_crossentropy', optimizer='RMSprop')



model.compile(loss='categorical_crossentropy', optimizer='Adam',metrics=['accuracy', 'top_k_categorical_accuracy'])
 
# Split the data

X_train_new, X_val_new, y_train_new,y_val_new =  train_test_split(X_train, Y_train, test_size=0.2, random_state=4)


# Train the model
earlystopping = EarlyStopping(patience=10, verbose=1)

hist = model.fit(X_train_new, y_train_new, validation_data=(X_val_new,y_val_new),
          batch_size=batch_size,nb_epoch = 200, callbacks=[earlystopping])



 # Evaluate the model
score = model.evaluate(X_val_new, y_val_new, batch_size=batch_size)
print('Test score:', score[0])
print('Test accuracy:', score[1])

In [ ]:
#X_train_new, X_val_new, y_train_new,y_val_new =  train_test_split(X_train, Y_train, test_size=0.2, random_state=4)
val_pred = []
val_actural = []
for video in X_val_new:
  print(video.shape)
  video = np.expand_dims(video, axis=0)
  pred = model.predict(video)
  #bashouf tele3 anhy class feehom
  val_pred.append(np.argmax(pred))

for label in y_val_new:
  print(label)
  val_actural.append(np.argmax(label))


from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn import metrics
from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sns

confusionMatrix = metrics.confusion_matrix(val_actural, val_pred)
confusionMatrixDataFrame = pd.DataFrame(confusionMatrix, index = ["Walking", "Tennis", "Jumping" , "Diving", "Basketball"], columns = ["Walking", "Tennis", "Jumping" , "Diving", "Basketball"])
plt.figure(figsize = (9, 9))
sns.heatmap(confusionMatrixDataFrame, annot = True)
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()

In [ ]:
import numpy as np
import os
videos = []
labels = []

for video in os.listdir(TEST_PATH):
    video_path = os.path.join(TEST_PATH, video)
    frames = []
    cap = cv2.VideoCapture(video_path)
    num_of_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    if(num_of_frames < 40):continue

    num = max(1, num_of_frames // 40)
    frame_idx = 0
    taken = 0
    while(1):
        ret, frame = cap.read()
        if(ret == False):
          break
        if frame_idx % num == 0:
            frame=cv2.resize(frame,(img_rows,img_cols),interpolation=cv2.INTER_AREA)
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            frames.append(gray)
            taken += 1
        if(taken == 40):
          break
        frame_idx += 1

    cap.release()
    cv2.destroyAllWindows()
    input=np.array(frames)
    input = np.expand_dims(input, axis=0)
    print(input.shape)
    input = input.transpose((2 , 3, 1, 0 ))
    print(input.shape)
    input = np.expand_dims(input, axis=0)

    prediction = model.predict(input)
    videos.append(video.replace('npy', 'mpg'))
    labels.append(np.argmax(prediction))
    print(video, np.argmax(prediction))


In [ ]:
categories_mapping = {0: 4,
                      1: 3,
                      2: 1,
                      3: 0,
                      4: 2
                      }

In [ ]:
with open('submit.csv','w') as file:
    file.write('Video,')
    file.write('Label')
    file.write('\n')
    for i in range(len(videos)):
        file.write(videos[i] + ',')
        file.write(str(categories_mapping[labels[i]]))
        file.write('\n')